In [ ]:
# Load the defaults
import numpy as np
import matplotlib.pyplot as plt
import pylab
import lmfit
from lmfit import Minimizer, Parameters, report_fit
plt.style.use("/raven/u/cmor/jupyter/HP_project/my.mplstyle")

In [ ]:
Lx_scan = [209.440, 104.720, 69.813, 52.360, 41.888, 34.907, 29.920, 26.180, 23.271, 
           20.944, 19.040, 17.453, 16.111, 14.960, 13.963, 13.090, 12.320, 11.636, 11.023, 
           10.472, 9.973, 9.520, 9.106, 8.727, 8.378, 8.055, 7.757, 7.480, 7.222, 6.981, 6.756]

kx = np.zeros_like(Lx_scan)
for n in range(0, len(Lx_scan)):
               kx[n] = 2*np.pi/Lx_scan[n]

In [ ]:
def gene_fit(t, A, g, w, d):  # This is what the diagnostics tool gives you
    return (A - d * t) + ((1 - (A - d * t)) * np.exp(-g * t) * np.cos(w * t))

def edi_fit(t, A, g, w, R):  # Edi's fit reported in Monreal 2016 
    return A * np.cos(w * t) * np.exp(-g * t) + R

def edi_2(t, a, b, c, d, e, g, k, m):  # Edi's fit (from mail) 
    return b * np.cos(2 * np.pi * c * t + d) * np.exp(- e * t) + a + m / (1 + k * t**g)

def idl_tabulate(x, f, p=5) :  # The function that will help Gao's analysis to integrate and get A
    def newton_cotes(x, f) :
        if x.shape[0] < 2 :
            return 0
        rn = (x.shape[0] - 1) * (x - x[0]) / (x[-1] - x[0])
        weights = scipy.integrate.newton_cotes(rn)[0]
        return (x[-1] - x[0]) / (x.shape[0] - 1) * np.dot(weights, f)
    ret = 0
    for idx in range(0, x.shape[0], p - 1) :
        ret += newton_cotes(x[idx:idx + p], f[idx:idx + p])
    return ret

In [ ]:
def resplot():
    for n in reversed(range(100, len(np.diff(a_orig)))):
        if np.abs(np.average(np.diff(a_orig[n-20:n+20]))) > 0.00004:
            break
    dif = np.diff(a_orig)

    plt.figure()
    plt.plot(t_orig, a_orig, linewidth=1)
    plt.scatter(t_orig[n], a_orig[n], s=15, c='red')
    plt.show()

    return

In [ ]:
def data_grab(t0, tf, n):

    if n <= 8:
        gam = pylab.loadtxt("timetraceions_000{0}.dat".format(n+1))
    elif n >=9 and n <= 98:
        gam = pylab.loadtxt("timetraceions_00{0}.dat".format(n+1))
    else:
        gam = pylab.loadtxt("timetraceions_0{0}.dat".format(n+1))

    a = np.divide(gam[:, 1], gam[0, 1], out=np.zeros_like(gam[:, 1]),
                  where=gam[0, 1] != 0)  # The real component
    b = np.divide(gam[:, 2], gam[0, 1], out=np.zeros_like(gam[:, 2]),
                  where=gam[0, 1] != 0)  # Imaginary component
    E = np.sqrt(a**2 + b**2)  # Modulus

    t = gam[:, 0]  # Time

    # Crop the simulation if it's too long. Specify that in the beginning of the function
    if t0 != None:
        ini = (np.abs(t - t0)).argmin()
    else:
        ini = (np.abs(t - t[0])).argmin()
    if tf != None:
        fin = (np.abs(t - tf)).argmin()
    else:
        fin = (np.abs(t - t[-1])).argmin()
    
    y = a[ini:fin]
    E = E[ini:fin]
    t = t[ini:fin]
    
    return y, t

In [ ]:
def scan_fitter(plot='real'):

    global Lx_scan, t0, tf # Get all this stuff needed
    
    all_a = []
    all_b = []
    all_c = []
    all_d = []
    all_e = []
    all_m = []
    all_g = []
    all_k = []
    all_x = []
    all_r = []
    
    ke = len(Lx_scan)
    
    if plot != None:
        plt.figure()
        plt.xlabel('Time units $(a/\\rho_{ti})$')
        plt.ylabel('Zonal Potential $(\\frac{\langle\phi_{zf}(t)\\rangle}{\langle\phi_{zf}(0)\\rangle})$')
        plt.axhline(y=0, color='black', linewidth=2, linestyle='dashed')  # Have a reference of zero
            
    ## Initial guesses:
    
    ## LHD_a0_1turn_0002
#     a = 0.01009373
#     b = 0.78488760
#     c = 0.05487475
#     d = 0.26603645
#     e = 0.03228965
#     g = 2.49670734
#     k = 0.00706999
#     m = 0.26191753

#     ## LHD_surface_0003  # Gets LHD
#     a = 0.02229146
#     b = 0.76549490
#     c = 0.05328876
#     d = 3.45263009
#     e = 0.03609705
#     g = 2.86842395
#     k = 0.00536408
#     m = 0.28425027
    
#     ## LHD_a0_1turn_0026
#     a = 0.04880500
#     b = 0.10539227
#     c = 0.01356965
#     d = 6.61571669
#     e = 0.02805173
#     g = 3.14075299
#     k = 0.01912469
#     m = 0.78544358

    # W7-X surface kx:0.5
    a = 0.01738757
    b = 0.58281430
    c = 0.00289424
    d = 12.1264916
    e = 0.00408636
    g = 1.82598761
    k = 0.02491807
    m = 0.44101235

    for n in range(0, ke):

        y, t = data_grab(t0, tf, n)

        gmodel = lmfit.Model(edi_2)
        if n == 0:
            pars = Parameters()
            pars.add('a', value=a)  # Residual
            pars.add('b', value=b)
            pars.add('c', value=c)
            pars.add('d', value=d)
            pars.add('e', value=e)  # decay
            pars.add('g', value=g)
            pars.add('k', value=k)
            pars.add('m', value=m)
        else:
            pars = Parameters()
            pars.add('a', value=all_a[-1])  #, min=0.0005, max=0.035)  # Residual
            pars.add('b', value=all_b[-1])
            pars.add('c', value=all_c[-1])
            pars.add('d', value=all_d[-1])
            pars.add('e', value=all_e[-1])  # decay
            pars.add('g', value=all_g[-1])
            pars.add('k', value=all_k[-1])
            pars.add('m', value=all_m[-1])

        result = gmodel.fit(y, pars, t=t, nan_policy='propagate')

        all_a = np.append(all_a, result.best_values.get("a"))
        all_b = np.append(all_b, result.best_values.get("b"))
        all_c = np.append(all_c, result.best_values.get("c"))
        all_d = np.append(all_d, result.best_values.get("d"))
        all_e = np.append(all_e, result.best_values.get("e"))
        all_m = np.append(all_m, result.best_values.get("m"))
        all_g = np.append(all_g, result.best_values.get("g"))
        all_k = np.append(all_k, result.best_values.get("k"))
        all_x = np.append(all_x, result.chisqr)
        all_r = np.append(all_r, 1 - result.residual.var() / np.var(y))

        if plot != None:
#             plt.plot(t, result.init_fit, label=r"Initial guess")
#             plt.plot(t, result.best_fit, 'r--', linewidth=1, label=r"Best fit", alpha=0.5)
            plt.plot(t, y, label='{0}'.format(Lx_scan[n]))
    if plot != None:
        plt.show()

    return all_x, all_r, all_a, all_e

In [1]:
def scan_fitter_reversed(plot='real'):

    global Lx_scan, t0, tf # Get all this stuff needed
    
    all_a = []
    all_b = []
    all_c = []
    all_d = []
    all_e = []
    all_m = []
    all_g = []
    all_k = []
    all_x = []
    all_r = []
    
    ke = len(Lx_scan)
    
    if plot != None:
        plt.figure()
        plt.xlabel('Time units $(a/\\rho_{ti})$')
        plt.ylabel('Zonal Potential $(\\frac{\langle\phi_{zf}(t)\\rangle}{\langle\phi_{zf}(0)\\rangle})$')
        plt.axhline(y=0, color='black', linewidth=2, linestyle='dashed')  # Have a reference of zero
            
    ## Initial guesses:
    
    ## LHD_a0_1turn_0026
#     a = 0.04880500
#     b = 0.10539227
#     c = 0.01356965
#     d = 6.61571669
#     e = 0.02805173
#     g = 3.14075299
#     k = 0.01912469
#     m = 0.78544358
    
    ## LHD_surf_0026
    a = 0.05126107
    b = 0.13020194
    c = 0.01299829
    d = 9.83095429
    e = 0.03139539
    g = 3.11837672
    k = 0.01949793
    m = 0.76578451
    
    # W7-X surface 0030
    a = 0.13439345
    b = 0.48977940
    c = 6.7678e-05
    d = 19.3305767
    e = 2.2336e-04
    g = 1.04559575
    k = 0.20279057
    m = 0.44645688

    for n in reversed(range(0, ke)):

        y, t = data_grab(t0, tf, n)

        gmodel = lmfit.Model(edi_2)
        if n == ke-1:
            pars = Parameters()
            pars.add('a', value=a)  # Residual
            pars.add('b', value=b)
            pars.add('c', value=c)
            pars.add('d', value=d)
            pars.add('e', value=e)  # decay
            pars.add('g', value=g)
            pars.add('k', value=k)
            pars.add('m', value=m)
            
        elif n <= 16:
            
            # W7-X surface kx:0.5
            a = 0.01738757
            b = 0.58281430
            c = 0.00289424
            d = 12.1264916
            e = 0.00408636
            g = 1.82598761
            k = 0.02491807
            m = 0.44101235
            
            pars = Parameters()
            pars.add('a', value=a)
            pars.add('b', value=b)
            pars.add('c', value=c)
            pars.add('d', value=d)
            pars.add('e', value=e)
            pars.add('g', value=g)
            pars.add('k', value=k)
            pars.add('m', value=m)
            
        else:
            pars = Parameters()
            pars.add('a', value=all_a[-1])  # Residual
            pars.add('b', value=all_b[-1])
            pars.add('c', value=all_c[-1])
            pars.add('d', value=all_d[-1])
            pars.add('e', value=all_e[-1])  # decay
            pars.add('g', value=all_g[-1])
            pars.add('k', value=all_k[-1])
            pars.add('m', value=all_m[-1])

        result = gmodel.fit(y, pars, t=t, nan_policy='propagate')

        all_a = np.append(all_a, result.best_values.get("a"))
        all_b = np.append(all_b, result.best_values.get("b"))
        all_c = np.append(all_c, result.best_values.get("c"))
        all_d = np.append(all_d, result.best_values.get("d"))
        all_e = np.append(all_e, result.best_values.get("e"))
        all_m = np.append(all_m, result.best_values.get("m"))
        all_g = np.append(all_g, result.best_values.get("g"))
        all_k = np.append(all_k, result.best_values.get("k"))
        all_x = np.append(all_x, result.chisqr)
        all_r = np.append(all_r, 1 - result.residual.var() / np.var(y))

        if plot != None:
#             plt.plot(t, result.init_fit, label=r"Initial guess")
#             plt.plot(t, result.best_fit, 'r--', linewidth=1, label=r"Best fit", alpha=0.5)
            plt.plot(t, y, label='{0}'.format(Lx_scan[n]))
    if plot != None:
        plt.show()
        
    all_a = np.flip(all_a)
    all_b = np.flip(all_b)
    all_c = np.flip(all_c)
    all_d = np.flip(all_d)
    all_e = np.flip(all_e)
    all_m = np.flip(all_m)
    all_g = np.flip(all_g)
    all_k = np.flip(all_k)
    all_x = np.flip(all_x)
    all_r = np.flip(all_r)

    return all_x, all_r, all_a, all_e

In [ ]:
def geometry(filename="../gist_0001"):

    geom = pylab.loadtxt("{0}".format(filename), skiprows=18)
    g11 = geom[:, 0]
    g12 = geom[:, 1]
    g13 = geom[:, 2]
    g22 = geom[:, 3]
    g23 = geom[:, 4]
    g33 = geom[:, 5]
    B = geom[:, 6]
    k_N = geom[:, 7]
    k_G = geom[:, 8]
    J = geom[:, 10]
        
    iota     = 1/data[7]
    B_ref    = data[2]
    B_00     = data[3]
    L = []
    
    ###########################################
    # Calculate the arc length of all the metrics:
    
    phi = z  # This is the toroidal angle
    
    phi_half = list()  # Will store the arc length from phi=0 in the positive direction
    phi_int = list()
    
    # Integrating the Jacobian with  pect to theta gives the arc length 
    # as a function of phi
    
    for t in range(len(phi)):
        phi_int.append(simps(J[:t+1], phi[:t+1]))  # This obtains the arc length from the element 1 of the array
    
    for t in range(len(phi[:65])):
        phi_half.append(simps(J[:t+1], phi[:t+1]))  # This obtains it from the outboard midplane (Half)
    
    phi_cent = np.concatenate((np.negative(phi_half[::-1][:-1]),  phi_half[:-1]))  # This mirrors and appends the arc length centered in zero
    dphi = np.diff(phi)
    
    ###########################################

    s = np.diff(g12 / g11) / np.diff(phi_cent)
    s = np.concatenate((s[:64], s[64:65], s[64:]), axis=0)  # Local shear calculation
    
    maxims, minis = peakdet(s, delta=1, x=phi_cent)  # Get the peaks observed in the local shear
    
    color = colourWheel[14]
    R_eff = 3.7282393
    L = abs(minis[0, 0] / 2)
    q_eff = L / (np.pi * R_eff)

    return R_eff, L, q_eff